In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.1 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Thu Aug 29 03:40:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
GENE_CHIN_DICTIONARY_PATH = "/content/drive/My Drive/szeyap/gene_chin_dictionary.json"
STEPHEN_LI_DICTIONARY = "/content/drive/My Drive/szeyap/stephen_li_dictionary.json"

In [ ]:
import json

with open(GENE_CHIN_DICTIONARY_PATH, 'r') as f:
  GC = json.load(f)[1:]
  for entry in GC:
    keys_need_join = ('TRAD', 'SIMP')
    for key in keys_need_join:
      if isinstance(entry[key], list) and entry[key] and entry[key][0]:
        entry.update({
          key: " or ".join((word if word is not None else "") for word in entry[key])
        })

with open(STEPHEN_LI_DICTIONARY, 'r') as f:
  SL = json.load(f)

GC_EN_sentences = [entry["DEFN"] for entry in GC]
SL_EN_sentences = [entry["english"] for entry in SL]

GC_CH_sentences = [entry["SIMP"] for entry in GC]
SL_CH_sentences = [entry["taishanese"] for entry in SL]

In [ ]:
from sentence_transformers import SentenceTransformer

LANG_MODEL = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device=device)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

In [ ]:
gc_en_embeddings = LANG_MODEL.encode(GC_EN_sentences, show_progress_bar=True)
gc_ch_embeddings = LANG_MODEL.encode(GC_CH_sentences, show_progress_bar=True)

Batches:   0%|          | 0/2558 [00:00<?, ?it/s]

Batches:   0%|          | 0/2558 [00:00<?, ?it/s]

In [ ]:
torch.save(gc_en_embeddings, "GENE_CHIN_EN_EMBEDDINGS.pt")
torch.save(gc_ch_embeddings, "GENE_CHIN_CH_EMBEDDINGS.pt")

In [ ]:
sl_en_embeddings = LANG_MODEL.encode(SL_EN_sentences, show_progress_bar=True)
sl_ch_embeddings = LANG_MODEL.encode(SL_CH_sentences, show_progress_bar=True)

Batches:   0%|          | 0/282 [00:00<?, ?it/s]

Batches:   0%|          | 0/282 [00:00<?, ?it/s]

In [ ]:
torch.save(sl_en_embeddings, "STEPHEN_LI_EN_EMBEDDINGS.pt")
torch.save(sl_ch_embeddings, "STEPHEN_LI_CH_EMBEDDINGS.pt")